In [1]:
from typing import Callable, Any

In [2]:
def ajax_call(func: Callable[[dict], Any]) -> Any:
    pass

In [19]:
def get_server_stuff(callback: Callable[[dict], Any]) -> Any:
    return ajax_call(lambda json: callback(json))

`ajax_call(lambda json: callback(json))` is the same as `ajax_call(callback)`. Now my `do_server_stuff` becomes -

In [20]:
def get_server_stuff(callback: Callable[[dict], Any]) -> Any:
    return ajax_call(callback)

Here essentially we are saying that $f(x) = g(x)$ which means that $f = g$, i.e., 

In [12]:
do_server_stuff = ajax_call

In general if I see a pattern where -
```python
func1 = lambda arg: func2(arg)
```

or equivalently -
```python
def func1(arg):
    return func2(arg)
```

Then I can simply assign `func2` to `func1` and be done with it -
```python
func1 = func2
```

In [17]:
length = lambda ary: return len(ary)

SyntaxError: invalid syntax (4587245.py, line 1)

In [14]:
length([1, 2, 3])

3

In [18]:
def length(ary: list[Any]) -> int:
    return len(ary)

length = lambda ary: len(ary)

In [21]:
ary = [1, 2, 3]

In [22]:
dir(ary)

['__add__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [23]:
from functools import partial

In [24]:
def double(x):
    return 2*x

In [25]:
double_all = partial(map, double)

In [26]:
list(double_all([1, 2, 3]))

[2, 4, 6]

In [29]:
class Cookie:
    def __init__(self, flavor, calories):
        self.flavor = flavor
        self.calories = calories

    def __or__(self, other):
        self.flavor += " " + other.flavor
        self.calories += other.calories
        return self

In [30]:
choclate_chip = Cookie("Chocolate Chip", 200)
snicker_doodle = Cookie("Snicker Doodle", 180)

In [31]:
tp = choclate_chip | snicker_doodle

In [32]:
tp

In [33]:
tp.flavor

'Chocolate Chip Snicker Doodle'

In [34]:
tp.calories

380

In [55]:
from functools import reduce

def compose(*funcs):
    return lambda x: reduce(lambda acc, func: func(acc), funcs, x)

def easy_compose(*funcs):
    def composed(x):
        acc = x
        for func in funcs:
            acc = func(acc)
        return acc
    
    return composed

In [46]:
def head(xs):
    return xs[0]

def reverse(xs):
    return xs[::-1]

In [47]:
tp = [1, 2, 3]

In [48]:
head(reverse(tp))

3

In [49]:
last = compose(reverse, head)

In [50]:
last([1, 2, 3])

3

In [56]:
last = easy_compose(reverse, head)

In [57]:
last([10, 20, 40])

40

In [88]:
class Func:
    def __init__(self, func):
        self._func = func

    def __call__(self, *args, **kwargs):
        return self._func(*args, **kwargs)
    
    def __or__(self, other_func):
        def new_func(*args, **kwargs):
            return other_func._func(self._func(*args, **kwargs))
        
        return Func(new_func)

In [89]:
def composable(func):
    return Func(func)

In [90]:
import math

@composable
def double(x):
    return 2 * x

@composable
def sqrt(x):
    return math.sqrt(x)

@composable
def halve(x):
    return x // 2

@composable
def square(x):
    return x**2

In [91]:
f = double | sqrt

In [92]:
f(8)

4.0

In [93]:
f2 = double | sqrt | halve

In [94]:
f2(8)

2.0

In [95]:
f3 = double | sqrt | halve | square

In [96]:
f3(8)

4.0

In [97]:
(double | sqrt | halve | square)(8)

4.0

In [98]:
square(halve(sqrt(double(8))))

4.0

```
(double(8) | sqrt | halve | square).value

8 | double | sqrt | halve | square
```

In [100]:
@composable
def reverse(xs):
    return xs[::-1]

@composable
def head(xs):
    return xs[0]

In [101]:
(reverse | head)([1, 2, 3])

3

In [102]:
last = reverse | head

In [103]:
last([1, 2, 3])

3

In [105]:
def add(x, y):
    return x + y

In [106]:
add_7 = partial(add, 7)

In [107]:
add_7(10)

17

In [108]:
def curry(func):
    def curried(x):
        return partial(func, x)
    return curried

In [109]:
adder = curry(add)

In [110]:
add_7 = adder(7)

In [111]:
add_7(10)

17

In [112]:
def gen_fibs(cap):
    i, j = 0, 1
    curr = 0
    while curr < cap:
        x = i + j
        yield x
        i, j = j, x
        curr += 1
print(list(gen_fibs(10)))

[1, 2, 3, 5, 8, 13, 21, 34, 55, 89]


In [113]:
chunks =  zip(*[iter(gen_fibs(10))]*3)
list(chunks)

[(1, 2, 3), (5, 8, 13), (21, 34, 55)]

In [114]:
type(gen_fibs(10))

generator

In [115]:
chunks = zip(*[gen_fibs(10)]*3)
list(chunks)

[(1, 2, 3), (5, 8, 13), (21, 34, 55)]

In [123]:
chunks_of_3 = partial(lambda n, it: zip(*[it]*n), 3)

In [125]:
list(chunks_of_3([1, 2, 3, 4, 5, 6]))

[(1, 1, 1), (2, 2, 2), (3, 3, 3), (4, 4, 4), (5, 5, 5), (6, 6, 6)]

In [130]:
chunks_of = lambda n, seq: zip(*[iter(seq)]*n)

In [132]:
list(chunks_of(3, [1, 2, 3, 4, 5, 6]))

[(1, 2, 3), (4, 5, 6)]

In [133]:
chunks_of_3 = partial(lambda n, seq: zip(*[iter(seq)]*n), 3)

In [135]:
list(chunks_of_3([1, 2, 3, 4, 5, 6]))

[(1, 2, 3), (4, 5, 6)]

In [137]:
list(chunks_of_3(range(15)))

[(0, 1, 2), (3, 4, 5), (6, 7, 8), (9, 10, 11), (12, 13, 14)]

```
r: [T] -> [T]
a: T -> U
map(a): [T] -> [U]
r | map(a) == map(a) | r

r | a*: [T] -> [U]
a* | r: [T] -> [U]
```

In [138]:
x = True
y = False
x or y

True